In [1]:
# Import dependencies
import pandas as pd
import numpy as np
from config import db_password
from sqlalchemy import create_engine
import psycopg2
import pandas.io.sql as sqlio
import sklearn as skl
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

C:\Users\mulle\anaconda3\envs\mlenv\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\mulle\anaconda3\envs\mlenv\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\mulle\anaconda3\envs\mlenv\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\mulle\anaconda3\envs\mlenv\lib\site-packages\tensorflow\python\f

In [2]:
#create database connection variable 
conn = psycopg2.connect(user="postgres", password=db_password, host="localhost", database="final_project")

In [3]:
#execute query and save it to a variable
query="select * from chess_data"
chess_df = sqlio.read_sql_query(query,conn)

In [4]:
chess_df.drop(['id'],axis=1, inplace=True)

In [5]:
# Split moves column into moves df
moves_df = chess_df['moves'].str.split(' ', n=10, expand=True)

# Drop column 10 and rename columns
moves_df=moves_df.drop(10,axis=1)
moves_df.columns= ["Wm1","Bm1","Wm2","Bm2","Wm3","Bm3","Wm4","Bm4","Wm5","Bm5"]

moves_df["outcome"] = chess_df["winner"]

# drop na
moves_df = moves_df.dropna()

In [7]:
# Changing moves to numbers
for col in moves_df.columns:
    print(col,end=' ')
    
    # Get list of unique values
    values = list(set(moves_df[col].values))
    
    # Create numerical dictionary
    values_with_indexes = {}
    for i, v in enumerate(values):
        values_with_indexes[v] = i
    
    # Replace column
    moves_df.replace({col: values_with_indexes},inplace=True)

moves_df.head()

Wm1 Bm1 Wm2 Bm2 Wm3 Bm3 Wm4 Bm4 Wm5 Bm5 outcome 

,Wm1,Bm1,Wm2,Bm2,Wm3,Bm3,Wm4,Bm4,Wm5,Bm5,outcome
0,15,4,50,75,191,48,425,226,4,557,1
1,15,12,22,48,156,203,257,106,130,353,0
2,16,2,62,67,20,215,51,179,139,223,1
3,15,4,5,73,181,199,267,387,533,616,1
4,16,2,5,67,229,329,212,193,47,202,1


In [ ]:
# Export csv containing sample data to be imported into ML
# moves_df.to_csv("ML_sample_data.csv")

In [8]:
# Split our preprocessed data into our features and target arrays
y = moves_df["outcome"].values
X = moves_df.drop("outcome",1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 1)

C:\Users\mulle\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers

In [10]:
# Create the keras sequential model
number_input_features = len(X_train_scaled[0])
nn_model = tf.keras.models.Sequential()

In [11]:
# Add the first layer including input layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid", input_dim = number_input_features))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [12]:
# Add the ouput layer that uses a probability activation function
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

In [13]:
# Create a summary to check the structure of the sequential model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 11        
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 2         
Total params: 13
Trainable params: 13
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compile the model and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [15]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=5)

Epoch 1/5
754754/754754 [==============================] - 9s 12us/sample - loss: 0.6681 - acc: 0.4963
Epoch 2/5
754754/754754 [==============================] - 9s 12us/sample - loss: 0.6664 - acc: 0.4972
Epoch 3/5
754754/754754 [==============================] - 9s 12us/sample - loss: 0.6664 - acc: 0.4972
Epoch 4/5
754754/754754 [==============================] - 9s 12us/sample - loss: 0.6664 - acc: 0.4972
Epoch 5/5
754754/754754 [==============================] - 9s 12us/sample - loss: 0.6664 - acc: 0.4972


In [16]:
#  now that our deep learning model is properly trained, we can evaluate the model's performance by testing its predictive capabilities on our testing dataset
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

251585/251585 - 1s - loss: 0.6728 - acc: 0.4992
Loss: 0.672755202671221, Accuracy: 0.49915534257888794
